In [1]:
from helper_dataprocessing import *
import warnings
warnings.filterwarnings('ignore') 
# https://us02web.zoom.us/j/7410331468?pwd=b21CSWloRURiTWVpQjVGd0V3ZXhGUT09

In [2]:
DATA_RESULTS_PATH = r"data/raw/results.txt"
DATA_TEST_PATH = r"data/raw/tests.txt"
DATA_SPECIES_PATH = "data/raw/species.txt"
DATA_PROPERTY_PATH = "data/cas_property_tot_new.csv"
keep_columns = ['obs_duration_mean', 'obs_duration_unit',
       'endpoint', 'effect', 'measurement', 'conc1_type', 'conc1_mean',
       'conc1_unit', 'test_cas', 'test_location', 'exposure_type',
       'control_type', 'media_type', 'application_freq_unit', 'class',
       'tax_order', 'family', 'genus', 'species','smiles','pubchem2d', 
       'melting_point', 'melting_unit', 'water_solubility',
       'mol_weight']

In [3]:
tests, species, results = load_raw_data(DATA_TEST_PATH, DATA_RESULTS_PATH, DATA_SPECIES_PATH)
cas_property = pd.read_csv(DATA_PROPERTY_PATH).drop(['Unnamed: 0'],axis=1)

tests loaded
species loaded
results loaded


<font size=5>Simple

In [7]:
results_prefilter = prefilter(species, tests, results,endpoint="LC50|EC50",label='simple',effect='BCM')
# fish_species ='Rainbow Trout'/ 'Fathead Minnow'
results_prefilter.loc[:, 'test_cas'] = results_prefilter.test_cas.apply(to_cas)
filtered_results = results_prefilter.merge(cas_property,on='test_cas')

There are 185540 tests.(only LC50|EC50)
There are 1072 tests consideing aboutBCM.
There are 2079 fish species in total.
There are 155208 tests on these fishes.
All merged into one dataframe. Size was (91, 272) .
The unique chemical number was 47 .


<font size=2> Generalize the concentration, observation time and delete the missing data

In [ ]:
best_db = select_impute_features(filtered_results,keep_columns)

Delete the repeated experiments

In [29]:
final_db = repeated_experiments(best_db)

Add chemical properties

In [ ]:
final = final_db.merge(cas_property, on='test_cas')
final = final[~final.smiles.isnull()]
final = final[~final.pubchem2d.isnull()]
chem_feat = process_chemicals(final)
chem_feat.drop(['mol_weight','melting_unit'],axis=1,inplace=True)

change concentration from gram to molecular weight

In [26]:
mol =False
if mol == True:
    chem_feat['conc1_mean'] = chem_feat['conc1_mean']*chem_feat['mol_weight']

In [27]:
chem_feat.drop(['melting_unit','mol_weight'],axis =1,inplace=True)

In [31]:
null_output_counts(chem_feat)

In [30]:
chem_feat.to_csv('data/LOEC/loec_processed.csv')

<font size=5>Datafusion

In [7]:
results_prefilter = prefilter(species, tests, results,'LC50|EC50',label= 'datafusion', all_property='itself')
results_prefilter.loc[:, 'test_cas'] = results_prefilter.test_cas.apply(to_cas)
filtered_results = results_prefilter.merge(cas_property,on='test_cas')

There are 426070 tests.(except LC50|EC50)
There are 2079 fish species in total.
There are 163402 tests on these fishes.
All merged into one dataframe. Size was (175414, 272) .
The unique chemical number was 4331 .


In [8]:
ct = pd.crosstab(filtered_results.effect, filtered_results.endpoint)
ct

endpoint,ATCN,BCF,BCFD,BMC02.5,BMC04,BMC05,BMC06,BMC08,BMC10,BMC11,...,IC01,IC10,IC20,IC25,IC50,IC80,LC50,LOEC,MATC,NOEC
effect,,,,,,,,,,,,,,,,,,,,,
ACC,0,9144,256,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2169,5,1562
AVO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,0,0,75,3,104
BCM,0,0,0,0,0,0,0,0,0,0,...,0,4,39,0,27,0,1,8816,6,10427
BEH,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,1,1353,11,2581
CEL,0,0,0,0,0,0,0,0,0,0,...,0,40,45,0,106,2,10,1466,0,1935
DVP,0,0,0,0,0,8,0,0,0,0,...,0,0,0,1,1,0,0,785,77,1193
ENZ,0,0,0,1,1,1,0,1,3,0,...,0,0,51,0,347,0,0,6631,6,7025
FDB,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,610,1,521
GEN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,0,0,8983,10,11337


In [14]:
ct = pd.crosstab(filtered_results.effect, filtered_results.endpoint)
dfr = pd.DataFrame()
for j in tqdm(ct.columns):
    for i in ct.index:
        if (ct.loc[i,j] >200):
            pp = filtered_results[filtered_results.endpoint == j]
            pp = pp[pp.effect == i]
            best_db = select_impute_features(pp,keep_columns)
            x = np.array(repeated_experiments(best_db).conc1_mean.unique())
            try:
                final_db = repeated_experiments(best_db)
                final_db.loc[:,'endpoint'] = j
                final_db.loc[:,'effect'] =i
                dfr = pd.concat([dfr, final_db])
            except:
                continue

100%|██████████| 38/38 [00:10<00:00,  3.55it/s]


In [16]:
mol=False
final = dfr.merge(cas_property,on='test_cas',how='left')
final = final[~final.smiles.isnull()]
final = final[~final.pubchem2d.isnull()]
chem_feat = process_chemicals(final)
if mol == True:
    chem_feat['conc1_mean'] = chem_feat['conc1_mean'] *chem_feat['mol_weight']
chem_feat = chem_feat.drop(['melting_unit','mol_weight'],axis=1)

Finding atom number...
Finding number of alone atoms...
Finding single bounds number...
Finding double bounds number...
Finding triple bounds number...
Finding ring number...
Finding mol number...
Finding morgan density...
Finding partition number (LogP)...
Finding number of OH group...
Finding melting point...
Finding water solubility...


In [ ]:
ct = pd.crosstab(chem_feat.effect, chem_feat.endpoint)
df = pd.DataFrame()
df_chem = chem_feat.copy()
old_df = 1
while not (df.equals(old_df)):
    old_df = df
    df = pd.DataFrame()
    for i in tqdm(ct.index):
        for j in ct.columns:
            if (ct.loc[i,j] >200):
                [ths_1,ths_2,ths_3,ths_4] = np.quantile(df_chem[df_chem.effect == i].conc1_mean, [.2,.4,.6,.8])
                pp = df_chem[df_chem.endpoint == j]
                pp = pp[pp.effect == i]
                x = np.array(pp.conc1_mean.unique())
                try:
                    if  np.any(x <= ths_1) & np.any((ths_1 <x) & (x<= ths_2)) & np.any((ths_2 < x) & (x<=ths_3))& np.any((ths_3 < x)&(x <=ths_4))&np.any(x > ths_4):
                            pp.loc[:,'endpoint'] = j
                            pp.loc[:,'effect'] = i
                            df = pd.concat([df, pp])
                except:
                    continue
            else:
                continue
    df_chem = df
    print(df.equals(old_df))

In [ ]:
ct

<font size =3>final save

In [ ]:
df.to_csv('data/LC50/lc50_processed_df_itself.csv')

<font size =3>select effect

In [19]:
df.effect.unique()

array(['ACC', 'BCM', 'BEH', 'ENZ', 'GEN', 'MOR'], dtype=object)

In [ ]:
# repeated for each effect
dfr = df[df.effect == 'GEN']

dfr.to_csv('data/LC50/lc50_processed_df_gen.csv')

find datafusion datasets for specific fish species

In [2]:
import pandas as pd
lc50 = pd.read_csv('data/LC50/lc50_processed.csv').drop(columns = 'Unnamed: 0')   
fish = pd.read_csv('data/LC50/lc50_processed_rainbow.csv').drop(columns = 'Unnamed: 0')   
# fish2 = pd.read_csv('data/LC50/lc50_processed_fathead.csv').drop(columns = 'Unnamed: 0')  

In [74]:
nofish = pd.concat([lc50, fish,fish2]).drop_duplicates(keep=False)
nofish.to_csv('data/LC50/lc50_processed_norainbowfathead.csv')